Can we back into the same adversary prediction abilities using only the sorted edge weights? I.e. if we just look at the top n edges in the filtration, is this good enough? Or do we actually use persistent homology effectively?

In [4]:
import os
import parse
import pickle
import copy
import math

import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import dionysus as dion
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import hamming, cosine
from sklearn import svm
from sklearn.model_selection import cross_val_score
import sklearn
import networkx as nx
import seaborn as sns

PERCENTILE = 99

from pt_activation.models.alexnet import AlexNet

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
COLORS = ['#12355b', '#ff6978']
EDGE_COLOR = '#272d2d'
PLT_LABELS = ['Unaltered', 'Adversarial']

In [6]:
def get_adv_info(filename):
    format_string = 'true-{}_adv-{}_sample-{}.npy'
    parsed = parse.parse(format_string, filename)
    return {'true class':int(parsed[0]), 'adv class':int(parsed[1]), 'sample':int(parsed[2])}

def read_adversaries(loc):
    ret = []
    for f in os.listdir(loc):
        if os.path.isfile(os.path.join(loc,f)) and f.find('.npy') != -1:
            adv = np.load(os.path.join(loc, f))
            info = get_adv_info(f)
            info['adversary'] = adv
            ret.append(info)
    return ret
    

In [7]:
adv_directory_loc = '/home/tgebhart/projects/pt_activation/logdir/adversaries/cifar/projected_gradient_descent/alexnet_cifar.pt'
adversaries = read_adversaries(adv_directory_loc)
adversaries = sorted(adversaries,  key=lambda k: k['sample'])

In [8]:
def create_sample_graph(f,tnms,wm):

    subgraphs = {}
    m = dion.homology_persistence(f)
    dgm = dion.init_diagrams(m,f)[0]
    for i,c in enumerate(m):
        if len(c) == 2:
            w = f[i].data
            if (tnms[f[c[0].index][0]],tnms[f[c[1].index][0]]) in wm:
                w = wm[(tnms[f[c[0].index][0]],tnms[f[c[1].index][0]])]
            elif (tnms[f[c[1].index][0]],tnms[f[c[0].index][0]]) in wm:
                w = wm[(tnms[f[c[1].index][0]],tnms[f[c[0].index][0]])]
#                 else:
#                     print((tnms[f[c[0].index][0]],tnms[f[c[1].index][0]]))
#                     raise Exception('NO WM!')
            if False: #tnms[f[c[0].index][0]] in subgraphs:
                subgraphs[tnms[f[c[0].index][0]]].add_edge(tnms[f[c[0].index][0]],tnms[f[c[1].index][0]], weight=w)
            else:
                eaten = False
                for k, v in subgraphs.items():
                    if v.has_node(tnms[f[c[0].index][0]]):
                        if tnms[f[c[1].index][0]] in subgraphs:
                            v.add_node(f[c[1].index][0])
#                                 subgraphs[k] = nx.union(v, subgraphs[tnms[f[c[1].index][0]]])
                        else:
                            v.add_edge(tnms[f[c[0].index][0]], tnms[f[c[1].index][0]], weight=w)
                        eaten = True
                        break
                if not eaten:
                    g = nx.Graph()
                    g.add_edge(tnms[f[c[0].index][0]], tnms[f[c[1].index][0]], weight=w)
                    subgraphs[tnms[f[c[0].index][0]]] = g
    
    return subgraphs, dgm, create_lifetimes(f,subgraphs,dgm,tnms)

def create_lifetimes(f, subgraphs, dgm, ids):
    lifetimes = {}
    for pt in dgm:
        k = ids[f[pt.data][0]] 
        if k in subgraphs.keys():
            if pt.death < float('inf'):
                lifetimes[k] = pt.birth - pt.death
            else:
                lifetimes[k] = pt.birth
    return lifetimes
    

In [9]:
def create_subgraphs(model, batch_size, up_to):
    device = torch.device("cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True}
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(root='../../data/cifar', train=False,
                        download=True, transform = transforms.Compose(
                        [transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])), 
                        batch_size=batch_size, shuffle=False)

    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    subgraphs = []
    diagrams = []
    lifetimes = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss = F.nll_loss(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            for s in range(data.shape[0]):
                this_hiddens = [hiddens[i][s] for i in range(len(hiddens))]
                print('Filtration: {}'.format(s+t))
                f, nm, wm = model.compute_dynamic_filtration2(data[s], this_hiddens, percentile=PERCENTILE, return_nm=True, absolute_value=True, input_layer=False)
                tnm = {v: k for k, v in nm.items()}
                sg, dg, lifetime = create_sample_graph(f, tnm, wm)
                row = {'loss':test_loss, 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0]}
                res_df.append(row)
                subgraphs.append(sg)
                diagrams.append(dg)
                lifetimes.append(lifetime)

            t += batch_size
            if t >= up_to:
                break

    return pd.DataFrame(res_df), subgraphs, diagrams, lifetimes


def create_adversary_subgraphs(model, batch_size, up_to, adversaries):
    device = torch.device("cpu")
    adv_images = torch.tensor(np.array([a['adversary'] for a in adversaries]))
    adv_labels = torch.tensor(np.array([a['true class'] for a in adversaries]))
    adv_samples = [a['sample'] for a in adversaries]
    
    print(adv_images.shape, adv_labels.shape)
    
    advs = torch.utils.data.TensorDataset(adv_images, adv_labels)
    test_loader = torch.utils.data.DataLoader(advs, batch_size=batch_size, shuffle=False)
    
    model.eval()
    test_loss = 0
    correct = 0
    t = 0
    res_df = []
    subgraphs = []
    diagrams = []
    lifetimes = []
    with torch.no_grad():
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output, hiddens = model(data, hiddens=True)
            test_loss = F.nll_loss(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for s in range(data.shape[0]):
                this_hiddens = [hiddens[i][s] for i in range(len(hiddens))]
                print('Filtration: {}'.format(s+t))
                f, nm, wm = model.compute_dynamic_filtration2(data[s], this_hiddens, percentile=PERCENTILE, return_nm=True, absolute_value=True, input_layer=False)
                tnm = {v: k for k, v in nm.items()}
                sg, dg, lifetime = create_sample_graph(f, tnm, wm)
                row = {'loss':test_loss, 'class':target.cpu().numpy()[s], 'prediction':pred.cpu().numpy()[s][0]}
                res_df.append(row)
                subgraphs.append(sg)
                diagrams.append(dg)
                lifetimes.append(lifetime)

            t += (batch_size)
            if t >= up_to:
                break

    return pd.DataFrame(res_df), subgraphs, diagrams, lifetimes

In [10]:
model_location = '/home/tgebhart/projects/pt_activation/logdir/models/alexnet_cifar.pt'
model = AlexNet()
model.load_state_dict(torch.load(model_location, map_location='cpu'))

In [12]:
res_df, sample_graphs, dgms, lifetimes = create_subgraphs(model, 1, 500)

Filtration: 0
filtration size 52676
Filtration: 1
filtration size 69019
Filtration: 2
filtration size 35621
Filtration: 3
filtration size 76419
Filtration: 4
filtration size 63063
Filtration: 5
filtration size 38854
Filtration: 6
filtration size 63745
Filtration: 7
filtration size 54862
Filtration: 8
filtration size 75657
Filtration: 9
filtration size 66299
Filtration: 10
filtration size 81093
Filtration: 11
filtration size 77782
Filtration: 12
filtration size 56424
Filtration: 13
filtration size 74440
Filtration: 14
filtration size 39990
Filtration: 15
filtration size 77578
Filtration: 16
filtration size 65254
Filtration: 17
filtration size 57303
Filtration: 18
filtration size 85852
Filtration: 19
filtration size 54410
Filtration: 20
filtration size 59929
Filtration: 21
filtration size 58309
Filtration: 22
filtration size 67514
Filtration: 23
filtration size 66688
Filtration: 24
filtration size 57722
Filtration: 25
filtration size 96710
Filtration: 26
filtration size 48293
Filtration:

filtration size 74287
Filtration: 220
filtration size 59482
Filtration: 221
filtration size 77863
Filtration: 222
filtration size 89106
Filtration: 223
filtration size 55641
Filtration: 224
filtration size 33342
Filtration: 225
filtration size 80439
Filtration: 226
filtration size 74301
Filtration: 227
filtration size 49328
Filtration: 228
filtration size 36881
Filtration: 229
filtration size 42565
Filtration: 230
filtration size 61891
Filtration: 231
filtration size 60576
Filtration: 232
filtration size 68973
Filtration: 233
filtration size 68126
Filtration: 234
filtration size 47326
Filtration: 235
filtration size 59558
Filtration: 236
filtration size 60683
Filtration: 237
filtration size 57880
Filtration: 238
filtration size 64401
Filtration: 239
filtration size 36768
Filtration: 240
filtration size 68111
Filtration: 241
filtration size 77189
Filtration: 242
filtration size 70473
Filtration: 243
filtration size 63321
Filtration: 244
filtration size 51630
Filtration: 245
filtration s

filtration size 58405
Filtration: 436
filtration size 61734
Filtration: 437
filtration size 93456
Filtration: 438
filtration size 51096
Filtration: 439
filtration size 55833
Filtration: 440
filtration size 98124
Filtration: 441
filtration size 69784
Filtration: 442
filtration size 69993
Filtration: 443
filtration size 56041
Filtration: 444
filtration size 73590
Filtration: 445
filtration size 71302
Filtration: 446
filtration size 76273
Filtration: 447
filtration size 64639
Filtration: 448
filtration size 63697
Filtration: 449
filtration size 89635
Filtration: 450
filtration size 71589
Filtration: 451
filtration size 78415
Filtration: 452
filtration size 64531
Filtration: 453
filtration size 87276
Filtration: 454
filtration size 61992
Filtration: 455
filtration size 40909
Filtration: 456
filtration size 59767
Filtration: 457
filtration size 63613
Filtration: 458
filtration size 65531
Filtration: 459
filtration size 73981
Filtration: 460
filtration size 53060
Filtration: 461
filtration s

filtration size 62077
Filtration: 652
filtration size 36622
Filtration: 653
filtration size 61299
Filtration: 654
filtration size 83386
Filtration: 655
filtration size 86881
Filtration: 656
filtration size 86865
Filtration: 657
filtration size 76590
Filtration: 658
filtration size 66954
Filtration: 659
filtration size 69300
Filtration: 660
filtration size 62362
Filtration: 661
filtration size 57763
Filtration: 662
filtration size 54923
Filtration: 663
filtration size 37268
Filtration: 664
filtration size 74134
Filtration: 665
filtration size 78325
Filtration: 666
filtration size 46537
Filtration: 667
filtration size 41053
Filtration: 668
filtration size 61135
Filtration: 669
filtration size 52751
Filtration: 670
filtration size 57030
Filtration: 671
filtration size 78358
Filtration: 672
filtration size 38762
Filtration: 673
filtration size 73366
Filtration: 674
filtration size 65971
Filtration: 675
filtration size 35933
Filtration: 676
filtration size 69108
Filtration: 677
filtration s

filtration size 69175
Filtration: 868
filtration size 61526
Filtration: 869
filtration size 77156
Filtration: 870
filtration size 57576
Filtration: 871
filtration size 87082
Filtration: 872
filtration size 64755
Filtration: 873
filtration size 67211
Filtration: 874
filtration size 52475
Filtration: 875
filtration size 75691
Filtration: 876
filtration size 97210
Filtration: 877
filtration size 87000
Filtration: 878
filtration size 82087
Filtration: 879
filtration size 72452
Filtration: 880
filtration size 50515
Filtration: 881
filtration size 61011
Filtration: 882
filtration size 55498
Filtration: 883
filtration size 55030
Filtration: 884
filtration size 70330
Filtration: 885
filtration size 53717
Filtration: 886
filtration size 70086
Filtration: 887
filtration size 71916
Filtration: 888
filtration size 78017
Filtration: 889
filtration size 39407
Filtration: 890
filtration size 66538
Filtration: 891
filtration size 62431
Filtration: 892
filtration size 76377
Filtration: 893
filtration s

filtration size 52632
Filtration: 1082
filtration size 61834
Filtration: 1083
filtration size 35686
Filtration: 1084
filtration size 83098
Filtration: 1085
filtration size 58141
Filtration: 1086
filtration size 63470
Filtration: 1087
filtration size 65042
Filtration: 1088
filtration size 58490
Filtration: 1089
filtration size 72476
Filtration: 1090
filtration size 56603
Filtration: 1091
filtration size 65087
Filtration: 1092
filtration size 56437
Filtration: 1093
filtration size 65647
Filtration: 1094
filtration size 71133
Filtration: 1095
filtration size 61171
Filtration: 1096
filtration size 65135
Filtration: 1097
filtration size 33544
Filtration: 1098
filtration size 63706
Filtration: 1099
filtration size 53159
Filtration: 1100
filtration size 64807
Filtration: 1101
filtration size 88254
Filtration: 1102
filtration size 92165
Filtration: 1103
filtration size 55044
Filtration: 1104
filtration size 49905
Filtration: 1105
filtration size 63854
Filtration: 1106
filtration size 66022
Fil

Filtration: 1292
filtration size 68408
Filtration: 1293
filtration size 66844
Filtration: 1294
filtration size 73104
Filtration: 1295
filtration size 35961
Filtration: 1296
filtration size 51838
Filtration: 1297
filtration size 76949
Filtration: 1298
filtration size 68866
Filtration: 1299
filtration size 73304
Filtration: 1300
filtration size 61622
Filtration: 1301
filtration size 56301
Filtration: 1302
filtration size 39496
Filtration: 1303
filtration size 83158
Filtration: 1304
filtration size 58228
Filtration: 1305
filtration size 41190
Filtration: 1306
filtration size 59299
Filtration: 1307
filtration size 57257
Filtration: 1308
filtration size 74271
Filtration: 1309
filtration size 60925
Filtration: 1310
filtration size 54259
Filtration: 1311
filtration size 57297
Filtration: 1312
filtration size 72518
Filtration: 1313
filtration size 57675
Filtration: 1314
filtration size 78951
Filtration: 1315
filtration size 66945
Filtration: 1316
filtration size 37019
Filtration: 1317
filtrati

filtration size 60708
Filtration: 1503
filtration size 58316
Filtration: 1504
filtration size 73856
Filtration: 1505
filtration size 44959
Filtration: 1506
filtration size 76627
Filtration: 1507
filtration size 45239
Filtration: 1508
filtration size 64223
Filtration: 1509
filtration size 71547
Filtration: 1510
filtration size 69388
Filtration: 1511
filtration size 71888
Filtration: 1512
filtration size 54412
Filtration: 1513
filtration size 67022
Filtration: 1514
filtration size 69503
Filtration: 1515
filtration size 45478
Filtration: 1516
filtration size 55012
Filtration: 1517
filtration size 78443
Filtration: 1518
filtration size 79195
Filtration: 1519
filtration size 65603
Filtration: 1520
filtration size 65893
Filtration: 1521
filtration size 72433
Filtration: 1522
filtration size 46965
Filtration: 1523
filtration size 70157
Filtration: 1524
filtration size 75492
Filtration: 1525
filtration size 85209
Filtration: 1526
filtration size 95842
Filtration: 1527
filtration size 52506
Fil

Filtration: 1713
filtration size 72349
Filtration: 1714
filtration size 62028
Filtration: 1715
filtration size 37447
Filtration: 1716
filtration size 38425
Filtration: 1717
filtration size 82597
Filtration: 1718
filtration size 75448
Filtration: 1719
filtration size 62539
Filtration: 1720
filtration size 79489
Filtration: 1721
filtration size 49802
Filtration: 1722
filtration size 53103
Filtration: 1723
filtration size 82261
Filtration: 1724
filtration size 73994
Filtration: 1725
filtration size 49585
Filtration: 1726
filtration size 84812
Filtration: 1727
filtration size 67160
Filtration: 1728
filtration size 37095
Filtration: 1729
filtration size 43501
Filtration: 1730
filtration size 65165
Filtration: 1731
filtration size 90773
Filtration: 1732
filtration size 65696
Filtration: 1733
filtration size 79294
Filtration: 1734
filtration size 68873
Filtration: 1735
filtration size 52573
Filtration: 1736
filtration size 32592
Filtration: 1737
filtration size 65069
Filtration: 1738
filtrati

filtration size 86192
Filtration: 1924
filtration size 61615
Filtration: 1925
filtration size 51578
Filtration: 1926
filtration size 78730
Filtration: 1927
filtration size 54403
Filtration: 1928
filtration size 68457
Filtration: 1929
filtration size 53434
Filtration: 1930
filtration size 69794
Filtration: 1931
filtration size 64549
Filtration: 1932
filtration size 69526
Filtration: 1933
filtration size 63787
Filtration: 1934
filtration size 38747
Filtration: 1935
filtration size 46459
Filtration: 1936
filtration size 76970
Filtration: 1937
filtration size 55946
Filtration: 1938
filtration size 57765
Filtration: 1939
filtration size 38114
Filtration: 1940
filtration size 51663
Filtration: 1941
filtration size 71990
Filtration: 1942
filtration size 61502
Filtration: 1943
filtration size 51001
Filtration: 1944
filtration size 81154
Filtration: 1945
filtration size 45054
Filtration: 1946
filtration size 67517
Filtration: 1947
filtration size 73724
Filtration: 1948
filtration size 59738
Fil

In [11]:
# adv_df, adv_sample_graphs, adv_dgms, adv_lifetimes = create_adversary_subgraphs(model, 1, 500, adversaries)

torch.Size([5219, 1, 28, 28]) torch.Size([5219])
Filtration: 0
filtration size 111134
Filtration: 1
filtration size 114453
Filtration: 2
filtration size 95263
Filtration: 3
filtration size 112005
Filtration: 4
filtration size 105487
Filtration: 5
filtration size 97000
Filtration: 6
filtration size 109073
Filtration: 7
filtration size 104332
Filtration: 8
filtration size 129397
Filtration: 9
filtration size 109219
Filtration: 10
filtration size 100888
Filtration: 11
filtration size 109069
Filtration: 12
filtration size 111723
Filtration: 13
filtration size 104858
Filtration: 14
filtration size 103970
Filtration: 15
filtration size 116055
Filtration: 16
filtration size 112362
Filtration: 17
filtration size 103085
Filtration: 18
filtration size 114580
Filtration: 19
filtration size 104225
Filtration: 20
filtration size 104692
Filtration: 21
filtration size 101559
Filtration: 22
filtration size 110681
Filtration: 23
filtration size 101750
Filtration: 24
filtration size 114798
Filtration: 2

filtration size 111532
Filtration: 213
filtration size 108869
Filtration: 214
filtration size 106380
Filtration: 215
filtration size 115788
Filtration: 216
filtration size 134803
Filtration: 217
filtration size 97787
Filtration: 218
filtration size 106853
Filtration: 219
filtration size 115426
Filtration: 220
filtration size 105195
Filtration: 221
filtration size 104985
Filtration: 222
filtration size 114940
Filtration: 223
filtration size 107960
Filtration: 224
filtration size 114898
Filtration: 225
filtration size 118991
Filtration: 226
filtration size 88997
Filtration: 227
filtration size 119181
Filtration: 228
filtration size 112752
Filtration: 229
filtration size 103681
Filtration: 230
filtration size 103012
Filtration: 231
filtration size 109514
Filtration: 232
filtration size 111690
Filtration: 233
filtration size 109707
Filtration: 234
filtration size 115644
Filtration: 235
filtration size 116102
Filtration: 236
filtration size 111983
Filtration: 237
filtration size 103046
Filt

filtration size 121807
Filtration: 424
filtration size 96660
Filtration: 425
filtration size 110152
Filtration: 426
filtration size 110483
Filtration: 427
filtration size 108976
Filtration: 428
filtration size 106984
Filtration: 429
filtration size 109498
Filtration: 430
filtration size 112069
Filtration: 431
filtration size 110363
Filtration: 432
filtration size 112822
Filtration: 433
filtration size 126435
Filtration: 434
filtration size 114533
Filtration: 435
filtration size 103243
Filtration: 436
filtration size 107881
Filtration: 437
filtration size 110297
Filtration: 438
filtration size 103436
Filtration: 439
filtration size 107931
Filtration: 440
filtration size 113322
Filtration: 441
filtration size 104744
Filtration: 442
filtration size 104079
Filtration: 443
filtration size 106661
Filtration: 444
filtration size 115408
Filtration: 445
filtration size 107840
Filtration: 446
filtration size 112620
Filtration: 447
filtration size 101414
Filtration: 448
filtration size 95828
Filt

filtration size 109678
Filtration: 635
filtration size 107392
Filtration: 636
filtration size 111531
Filtration: 637
filtration size 63357
Filtration: 638
filtration size 101777
Filtration: 639
filtration size 100626
Filtration: 640
filtration size 52285
Filtration: 641
filtration size 89744
Filtration: 642
filtration size 106321
Filtration: 643
filtration size 114450
Filtration: 644
filtration size 115312
Filtration: 645
filtration size 108995
Filtration: 646
filtration size 57855
Filtration: 647
filtration size 110554
Filtration: 648
filtration size 106885
Filtration: 649
filtration size 104951
Filtration: 650
filtration size 106209
Filtration: 651
filtration size 99678
Filtration: 652
filtration size 105511
Filtration: 653
filtration size 115442
Filtration: 654
filtration size 103047
Filtration: 655
filtration size 91087
Filtration: 656
filtration size 100174
Filtration: 657
filtration size 110955
Filtration: 658
filtration size 112901
Filtration: 659
filtration size 110156
Filtrati

Filtration: 846
filtration size 85386
Filtration: 847
filtration size 96209
Filtration: 848
filtration size 118298
Filtration: 849
filtration size 68385
Filtration: 850
filtration size 105905
Filtration: 851
filtration size 103293
Filtration: 852
filtration size 118213
Filtration: 853
filtration size 111843
Filtration: 854
filtration size 110656
Filtration: 855
filtration size 106582
Filtration: 856
filtration size 119292
Filtration: 857
filtration size 95782
Filtration: 858
filtration size 117808
Filtration: 859
filtration size 106028
Filtration: 860
filtration size 107283
Filtration: 861
filtration size 110443
Filtration: 862
filtration size 110367
Filtration: 863
filtration size 114861
Filtration: 864
filtration size 109571
Filtration: 865
filtration size 112823
Filtration: 866
filtration size 98686
Filtration: 867
filtration size 96234
Filtration: 868
filtration size 119334
Filtration: 869
filtration size 108593
Filtration: 870
filtration size 103459
Filtration: 871
filtration size

Filtration: 1056
filtration size 104089
Filtration: 1057
filtration size 106630
Filtration: 1058
filtration size 66636
Filtration: 1059
filtration size 96947
Filtration: 1060
filtration size 109996
Filtration: 1061
filtration size 112171
Filtration: 1062
filtration size 105191
Filtration: 1063
filtration size 92568
Filtration: 1064
filtration size 108204
Filtration: 1065
filtration size 113220
Filtration: 1066
filtration size 105821
Filtration: 1067
filtration size 106411
Filtration: 1068
filtration size 112569
Filtration: 1069
filtration size 110649
Filtration: 1070
filtration size 91379
Filtration: 1071
filtration size 95866
Filtration: 1072
filtration size 110731
Filtration: 1073
filtration size 103691
Filtration: 1074
filtration size 95065
Filtration: 1075
filtration size 105266
Filtration: 1076
filtration size 110596
Filtration: 1077
filtration size 101705
Filtration: 1078
filtration size 106038
Filtration: 1079
filtration size 107772
Filtration: 1080
filtration size 113012
Filtra

Filtration: 1262
filtration size 112494
Filtration: 1263
filtration size 96642
Filtration: 1264
filtration size 116621
Filtration: 1265
filtration size 99480
Filtration: 1266
filtration size 111175
Filtration: 1267
filtration size 99999
Filtration: 1268
filtration size 99415
Filtration: 1269
filtration size 99179
Filtration: 1270
filtration size 110635
Filtration: 1271
filtration size 111303
Filtration: 1272
filtration size 106164
Filtration: 1273
filtration size 114573
Filtration: 1274
filtration size 100429
Filtration: 1275
filtration size 101079
Filtration: 1276
filtration size 93206
Filtration: 1277
filtration size 113798
Filtration: 1278
filtration size 58573
Filtration: 1279
filtration size 101669
Filtration: 1280
filtration size 115808
Filtration: 1281
filtration size 111133
Filtration: 1282
filtration size 104605
Filtration: 1283
filtration size 113732
Filtration: 1284
filtration size 52947
Filtration: 1285
filtration size 111614
Filtration: 1286
filtration size 99469
Filtratio

Filtration: 1468
filtration size 103423
Filtration: 1469
filtration size 121244
Filtration: 1470
filtration size 108437
Filtration: 1471
filtration size 99268
Filtration: 1472
filtration size 95008
Filtration: 1473
filtration size 113240
Filtration: 1474
filtration size 97787
Filtration: 1475
filtration size 105088
Filtration: 1476
filtration size 97883
Filtration: 1477
filtration size 106572
Filtration: 1478
filtration size 110069
Filtration: 1479
filtration size 119787
Filtration: 1480
filtration size 120796
Filtration: 1481
filtration size 111048
Filtration: 1482
filtration size 111542
Filtration: 1483
filtration size 108496
Filtration: 1484
filtration size 110977
Filtration: 1485
filtration size 116575
Filtration: 1486
filtration size 110915
Filtration: 1487
filtration size 94085
Filtration: 1488
filtration size 102775
Filtration: 1489
filtration size 115877
Filtration: 1490
filtration size 118726
Filtration: 1491
filtration size 102416
Filtration: 1492
filtration size 114607
Filtr

Filtration: 1674
filtration size 113866
Filtration: 1675
filtration size 108981
Filtration: 1676
filtration size 117199
Filtration: 1677
filtration size 95597
Filtration: 1678
filtration size 109948
Filtration: 1679
filtration size 89226
Filtration: 1680
filtration size 101246
Filtration: 1681
filtration size 77006
Filtration: 1682
filtration size 113559
Filtration: 1683
filtration size 101971
Filtration: 1684
filtration size 103075
Filtration: 1685
filtration size 99962
Filtration: 1686
filtration size 104796
Filtration: 1687
filtration size 99572
Filtration: 1688
filtration size 112394
Filtration: 1689
filtration size 95829
Filtration: 1690
filtration size 123592
Filtration: 1691
filtration size 107336
Filtration: 1692
filtration size 90066
Filtration: 1693
filtration size 118228
Filtration: 1694
filtration size 91179
Filtration: 1695
filtration size 108106
Filtration: 1696
filtration size 112576
Filtration: 1697
filtration size 108677
Filtration: 1698
filtration size 104003
Filtrati

Filtration: 1880
filtration size 99908
Filtration: 1881
filtration size 105684
Filtration: 1882
filtration size 106170
Filtration: 1883
filtration size 113963
Filtration: 1884
filtration size 104882
Filtration: 1885
filtration size 130033
Filtration: 1886
filtration size 109240
Filtration: 1887
filtration size 103924
Filtration: 1888
filtration size 103918
Filtration: 1889
filtration size 114914
Filtration: 1890
filtration size 105704
Filtration: 1891
filtration size 107671
Filtration: 1892
filtration size 112744
Filtration: 1893
filtration size 131022
Filtration: 1894
filtration size 115730
Filtration: 1895
filtration size 105420
Filtration: 1896
filtration size 108102
Filtration: 1897
filtration size 100108
Filtration: 1898
filtration size 112241
Filtration: 1899
filtration size 110164
Filtration: 1900
filtration size 98652
Filtration: 1901
filtration size 117954
Filtration: 1902
filtration size 107819
Filtration: 1903
filtration size 90103
Filtration: 1904
filtration size 115006
Fil

In [13]:
# with open(os.path.join(adv_directory_loc, 'prediction_adv_samples.pkl'), 'wb') as f:
#     pickle.dump(adv_sample_graphs, f)
with open(os.path.join(adv_directory_loc, 'prediction_samples.pkl'), 'wb') as f:
    pickle.dump(sample_graphs, f)

# sample_graphs = pickle.load( open(os.path.join(adv_directory_loc, 'samples.pkl'), "rb") )
# adv_sample_graphs = pickle.load( open(os.path.join(adv_directory_loc, 'adv_samples.pkl'), "rb") )

In [ ]:
# for dgm in dgms:
#     dion.plot.plot_diagram(dgm, show=True)

In [ ]:
# for adv_dgm in adv_dgms:
#     dion.plot.plot_diagram(adv_dgm, show=True)

In [ ]:
# thru = 3
# all_gois = []
# for i in range(len(sample_graphs)):
#     print(i)
#     ks = list(sample_graphs[i].keys())
#     a = [sample_graphs[i][k] for k in ks[:thru]]
#     all_gois.append(nx.compose_all(a))
    
# adv_all_gois = []
# for i in range(len(adv_sample_graphs)):
#     print(i)
#     ks = list(adv_sample_graphs[i].keys())
#     a = [adv_sample_graphs[i][k] for k in ks[:thru]]
#     adv_all_gois.append(nx.compose_all(a))

In [14]:
def hamming_distance(g1, g2, ret_labels=False):
    nodeset = set(list(g1.nodes) + list(g2.nodes))
    g1_vec = np.zeros((len(nodeset)))
    g2_vec = np.zeros((len(nodeset)))
    nodesetlist = list(nodeset)
    for i in range(len(nodesetlist)):
        node = nodesetlist[i]
        if node in g1.nodes:
            g1_vec[i] = 1.0
        if node in g2.nodes:
            g2_vec[i] = 1.0
    if ret_labels:
        return hamming(g1_vec, g2_vec), nodesetlist
    else:
        return hamming(g1_vec, g2_vec)
    
def edge_hamming_distance(g1, g2, ret_labels=False):
    edgeset = set(list(g1.edges) + list(g2.edges))
    g1_vec = np.zeros((len(edgeset)))
    g2_vec = np.zeros((len(edgeset)))
    edgesetlist = list(edgeset)
    for i in range(len(edgesetlist)):
        edge = edgesetlist[i]
        if edge in g1.edges:
            g1_vec[i] = 1.0
        if edge in g2.edges:
            g2_vec[i] = 1.0
    if ret_labels:
        return hamming(g1_vec, g2_vec), edgesetlist
    else:
        return hamming(g1_vec, g2_vec)
    
def lifetime_weighted_edge_distance(subgraphs1,subgraphs2,lifetimes1,lifetimes2,ret_labels=False):
    edges1 = {}
    edges2 = {}
    sg1keys = list(subgraphs1.keys())
    sg2keys = list(subgraphs2.keys())
    lifetimes1 = list(lifetimes1.values())
    lifetimes2 = list(lifetimes2.values())
    ml1 = max(lifetimes1)
    ml2 = max(lifetimes2)
    for i in range(len(sg1keys)):
        g = subgraphs1[sg1keys[i]]
        for e in g.edges:
            edges1[e] = lifetimes1[i]/ml1
    for i in range(len(sg2keys)):
        g = subgraphs2[sg2keys[i]]
        for e in g.edges:
            edges2[e] = lifetimes2[i]/ml2
    edgeset = set(list(edges1.keys()) + list(edges2.keys()))
    g1_vec = np.zeros((len(edgeset)))
    g2_vec = np.zeros((len(edgeset)))
    edgesetlist = list(edgeset)
    for i in range(len(edgesetlist)):
        edge = edgesetlist[i]
        if edge in edges1:
            g1_vec[i] += edges1[edge]
        if edge in edges2:
            g2_vec[i] += edges2[edge]
    if ret_labels:
        return cosine(g1_vec, g2_vec), edgesetlist
    else:
        return cosine(g1_vec, g2_vec)
    
def weighted_edge_distance(g1, g2, ret_labels=False):
    edgeset = set(list(g1.edges) + list(g2.edges))
    g1_vec = np.zeros((len(edgeset)))
    g2_vec = np.zeros((len(edgeset)))
    edgesetlist = list(edgeset)
    for i in range(len(edgesetlist)):
        edge = edgesetlist[i]
        if edge in g1.edges:
            g1_vec[i] = g1[edge[0]][edge[1]]['weight']
        if edge in g2.edges:
            g2_vec[i] = g2[edge[0]][edge[1]]['weight']
    if ret_labels:
        return cosine(g1_vec, g2_vec), edgesetlist
    else:
        return cosine(g1_vec, g2_vec)
    
def compute_kernel_matrix(gs, ls, take=3):
    ret = np.zeros((len(gs),len(gs)))
    for i in range(len(gs)):
        print('row: ', i)
        g1 = gs[i]
        lt1 = ls[i]
        g1keys = list(g1.keys())
        g1 = {k: g1[k] for k in g1keys[:take]}
        for j in range(len(gs)):
            g2 = gs[j]
            lt2 = ls[j]
            g2keys = list(g2.keys())
            g2 = {k: g2[k] for k in g2keys[:take]}
            ret[i,j] = 1 - lifetime_weighted_edge_distance(g1,g2,lt1,lt2)
    return ret

In [15]:
gs = sample_graphs
ls = lifetimes

In [16]:
X = compute_kernel_matrix(gs, ls, take=-1)
y = res_df['class'].values

row:  0
row:  1
row:  2
row:  3
row:  4
row:  5
row:  6
row:  7
row:  8
row:  9
row:  10
row:  11
row:  12
row:  13
row:  14
row:  15
row:  16
row:  17
row:  18
row:  19
row:  20
row:  21
row:  22
row:  23
row:  24
row:  25
row:  26
row:  27
row:  28
row:  29
row:  30
row:  31
row:  32
row:  33
row:  34
row:  35
row:  36
row:  37
row:  38
row:  39
row:  40
row:  41
row:  42
row:  43
row:  44
row:  45
row:  46
row:  47
row:  48
row:  49
row:  50
row:  51
row:  52
row:  53
row:  54
row:  55
row:  56
row:  57
row:  58
row:  59
row:  60
row:  61
row:  62
row:  63
row:  64
row:  65
row:  66
row:  67
row:  68
row:  69
row:  70
row:  71
row:  72
row:  73
row:  74
row:  75
row:  76
row:  77
row:  78
row:  79
row:  80
row:  81
row:  82
row:  83
row:  84
row:  85
row:  86
row:  87
row:  88
row:  89
row:  90
row:  91
row:  92
row:  93
row:  94
row:  95
row:  96
row:  97
row:  98
row:  99
row:  100
row:  101
row:  102
row:  103
row:  104
row:  105
row:  106
row:  107
row:  108
row:  109
row:  110


row:  831
row:  832
row:  833
row:  834
row:  835
row:  836
row:  837
row:  838
row:  839
row:  840
row:  841
row:  842
row:  843
row:  844
row:  845
row:  846
row:  847
row:  848
row:  849
row:  850
row:  851
row:  852
row:  853
row:  854
row:  855
row:  856
row:  857
row:  858
row:  859
row:  860
row:  861
row:  862
row:  863
row:  864
row:  865
row:  866
row:  867
row:  868
row:  869
row:  870
row:  871
row:  872
row:  873
row:  874
row:  875
row:  876
row:  877
row:  878
row:  879
row:  880
row:  881
row:  882
row:  883
row:  884
row:  885
row:  886
row:  887
row:  888
row:  889
row:  890
row:  891
row:  892
row:  893
row:  894
row:  895
row:  896
row:  897
row:  898
row:  899
row:  900
row:  901
row:  902
row:  903
row:  904
row:  905
row:  906
row:  907
row:  908
row:  909
row:  910
row:  911
row:  912
row:  913
row:  914
row:  915
row:  916
row:  917
row:  918
row:  919
row:  920
row:  921
row:  922
row:  923
row:  924
row:  925
row:  926
row:  927
row:  928
row:  929
row:  930


row:  1592
row:  1593
row:  1594
row:  1595
row:  1596
row:  1597
row:  1598
row:  1599
row:  1600
row:  1601
row:  1602
row:  1603
row:  1604
row:  1605
row:  1606
row:  1607
row:  1608
row:  1609
row:  1610
row:  1611
row:  1612
row:  1613
row:  1614
row:  1615
row:  1616
row:  1617
row:  1618
row:  1619
row:  1620
row:  1621
row:  1622
row:  1623
row:  1624
row:  1625
row:  1626
row:  1627
row:  1628
row:  1629
row:  1630
row:  1631
row:  1632
row:  1633
row:  1634
row:  1635
row:  1636
row:  1637
row:  1638
row:  1639
row:  1640
row:  1641
row:  1642
row:  1643
row:  1644
row:  1645
row:  1646
row:  1647
row:  1648
row:  1649
row:  1650
row:  1651
row:  1652
row:  1653
row:  1654
row:  1655
row:  1656
row:  1657
row:  1658
row:  1659
row:  1660
row:  1661
row:  1662
row:  1663
row:  1664
row:  1665
row:  1666
row:  1667
row:  1668
row:  1669
row:  1670
row:  1671
row:  1672
row:  1673
row:  1674
row:  1675
row:  1676
row:  1677
row:  1678
row:  1679
row:  1680
row:  1681
row:  1682

In [17]:
clf = svm.SVC(decision_function_shape='ovo', kernel='precomputed')
cross_val_score(clf, X, y, cv=10)

array([0.87254902, 0.89705882, 0.85714286, 0.87192118, 0.85074627,
       0.87437186, 0.83919598, 0.88324873, 0.83163265, 0.85051546])

In [19]:
np.average(cross_val_score(clf, X, y, cv=10))

0.8628382838342036

In [18]:
print('Natural performance: ', res_df[res_df['class'] == res_df['prediction']].shape[0]/res_df.shape[0])

Natural performance:  0.9695


In [27]:
take = 3
edges = set()
for i in range(len(sample_graphs)):
    for k in list(sample_graphs[i].keys())[:take]:
        for x in sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
            edges.add(edge_name)
            
edf = pd.DataFrame(np.zeros((len(sample_graphs),len(edges))), columns=list(edges))
for i in range(len(sample_graphs)):
    print('Sample: {}/{}'.format(i,len(sample_graphs)))
    lst = list(sample_graphs[i].keys())
    for k in lst[:take]:
        for x in sample_graphs[i][k].edges(data=True):
            edge_name = str(x[0])+'-'+str(x[1])
#             edf.iloc[i][edge_name] += x[2]['weight']
            edf.iloc[i][edge_name] += 1

Sample: 0/2000
Sample: 1/2000
Sample: 2/2000
Sample: 3/2000
Sample: 4/2000
Sample: 5/2000
Sample: 6/2000
Sample: 7/2000
Sample: 8/2000
Sample: 9/2000
Sample: 10/2000
Sample: 11/2000
Sample: 12/2000
Sample: 13/2000
Sample: 14/2000
Sample: 15/2000
Sample: 16/2000
Sample: 17/2000
Sample: 18/2000
Sample: 19/2000
Sample: 20/2000
Sample: 21/2000
Sample: 22/2000
Sample: 23/2000
Sample: 24/2000
Sample: 25/2000
Sample: 26/2000
Sample: 27/2000
Sample: 28/2000
Sample: 29/2000
Sample: 30/2000
Sample: 31/2000
Sample: 32/2000
Sample: 33/2000
Sample: 34/2000
Sample: 35/2000
Sample: 36/2000
Sample: 37/2000
Sample: 38/2000
Sample: 39/2000
Sample: 40/2000
Sample: 41/2000
Sample: 42/2000
Sample: 43/2000
Sample: 44/2000
Sample: 45/2000
Sample: 46/2000
Sample: 47/2000
Sample: 48/2000
Sample: 49/2000
Sample: 50/2000
Sample: 51/2000
Sample: 52/2000
Sample: 53/2000
Sample: 54/2000
Sample: 55/2000
Sample: 56/2000
Sample: 57/2000
Sample: 58/2000
Sample: 59/2000
Sample: 60/2000
Sample: 61/2000
Sample: 62/2000
Sa

Sample: 492/2000
Sample: 493/2000
Sample: 494/2000
Sample: 495/2000
Sample: 496/2000
Sample: 497/2000
Sample: 498/2000
Sample: 499/2000
Sample: 500/2000
Sample: 501/2000
Sample: 502/2000
Sample: 503/2000
Sample: 504/2000
Sample: 505/2000
Sample: 506/2000
Sample: 507/2000
Sample: 508/2000
Sample: 509/2000
Sample: 510/2000
Sample: 511/2000
Sample: 512/2000
Sample: 513/2000
Sample: 514/2000
Sample: 515/2000
Sample: 516/2000
Sample: 517/2000
Sample: 518/2000
Sample: 519/2000
Sample: 520/2000
Sample: 521/2000
Sample: 522/2000
Sample: 523/2000
Sample: 524/2000
Sample: 525/2000
Sample: 526/2000
Sample: 527/2000
Sample: 528/2000
Sample: 529/2000
Sample: 530/2000
Sample: 531/2000
Sample: 532/2000
Sample: 533/2000
Sample: 534/2000
Sample: 535/2000
Sample: 536/2000
Sample: 537/2000
Sample: 538/2000
Sample: 539/2000
Sample: 540/2000
Sample: 541/2000
Sample: 542/2000
Sample: 543/2000
Sample: 544/2000
Sample: 545/2000
Sample: 546/2000
Sample: 547/2000
Sample: 548/2000
Sample: 549/2000
Sample: 550/20

Sample: 976/2000
Sample: 977/2000
Sample: 978/2000
Sample: 979/2000
Sample: 980/2000
Sample: 981/2000
Sample: 982/2000
Sample: 983/2000
Sample: 984/2000
Sample: 985/2000
Sample: 986/2000
Sample: 987/2000
Sample: 988/2000
Sample: 989/2000
Sample: 990/2000
Sample: 991/2000
Sample: 992/2000
Sample: 993/2000
Sample: 994/2000
Sample: 995/2000
Sample: 996/2000
Sample: 997/2000
Sample: 998/2000
Sample: 999/2000
Sample: 1000/2000
Sample: 1001/2000
Sample: 1002/2000
Sample: 1003/2000
Sample: 1004/2000
Sample: 1005/2000
Sample: 1006/2000
Sample: 1007/2000
Sample: 1008/2000
Sample: 1009/2000
Sample: 1010/2000
Sample: 1011/2000
Sample: 1012/2000
Sample: 1013/2000
Sample: 1014/2000
Sample: 1015/2000
Sample: 1016/2000
Sample: 1017/2000
Sample: 1018/2000
Sample: 1019/2000
Sample: 1020/2000
Sample: 1021/2000
Sample: 1022/2000
Sample: 1023/2000
Sample: 1024/2000
Sample: 1025/2000
Sample: 1026/2000
Sample: 1027/2000
Sample: 1028/2000
Sample: 1029/2000
Sample: 1030/2000
Sample: 1031/2000
Sample: 1032/200

Sample: 1436/2000
Sample: 1437/2000
Sample: 1438/2000
Sample: 1439/2000
Sample: 1440/2000
Sample: 1441/2000
Sample: 1442/2000
Sample: 1443/2000
Sample: 1444/2000
Sample: 1445/2000
Sample: 1446/2000
Sample: 1447/2000
Sample: 1448/2000
Sample: 1449/2000
Sample: 1450/2000
Sample: 1451/2000
Sample: 1452/2000
Sample: 1453/2000
Sample: 1454/2000
Sample: 1455/2000
Sample: 1456/2000
Sample: 1457/2000
Sample: 1458/2000
Sample: 1459/2000
Sample: 1460/2000
Sample: 1461/2000
Sample: 1462/2000
Sample: 1463/2000
Sample: 1464/2000
Sample: 1465/2000
Sample: 1466/2000
Sample: 1467/2000
Sample: 1468/2000
Sample: 1469/2000
Sample: 1470/2000
Sample: 1471/2000
Sample: 1472/2000
Sample: 1473/2000
Sample: 1474/2000
Sample: 1475/2000
Sample: 1476/2000
Sample: 1477/2000
Sample: 1478/2000
Sample: 1479/2000
Sample: 1480/2000
Sample: 1481/2000
Sample: 1482/2000
Sample: 1483/2000
Sample: 1484/2000
Sample: 1485/2000
Sample: 1486/2000
Sample: 1487/2000
Sample: 1488/2000
Sample: 1489/2000
Sample: 1490/2000
Sample: 14

Sample: 1894/2000
Sample: 1895/2000
Sample: 1896/2000
Sample: 1897/2000
Sample: 1898/2000
Sample: 1899/2000
Sample: 1900/2000
Sample: 1901/2000
Sample: 1902/2000
Sample: 1903/2000
Sample: 1904/2000
Sample: 1905/2000
Sample: 1906/2000
Sample: 1907/2000
Sample: 1908/2000
Sample: 1909/2000
Sample: 1910/2000
Sample: 1911/2000
Sample: 1912/2000
Sample: 1913/2000
Sample: 1914/2000
Sample: 1915/2000
Sample: 1916/2000
Sample: 1917/2000
Sample: 1918/2000
Sample: 1919/2000
Sample: 1920/2000
Sample: 1921/2000
Sample: 1922/2000
Sample: 1923/2000
Sample: 1924/2000
Sample: 1925/2000
Sample: 1926/2000
Sample: 1927/2000
Sample: 1928/2000
Sample: 1929/2000
Sample: 1930/2000
Sample: 1931/2000
Sample: 1932/2000
Sample: 1933/2000
Sample: 1934/2000
Sample: 1935/2000
Sample: 1936/2000
Sample: 1937/2000
Sample: 1938/2000
Sample: 1939/2000
Sample: 1940/2000
Sample: 1941/2000
Sample: 1942/2000
Sample: 1943/2000
Sample: 1944/2000
Sample: 1945/2000
Sample: 1946/2000
Sample: 1947/2000
Sample: 1948/2000
Sample: 19

In [28]:
edf.head()

,"(2, 0, 14)-(1, 0, 2607)","(2, 0, 43)-(1, 0, 921)","(2, 0, 37)-(1, 0, 179)","(2, 0, 49)-(1, 0, 176)","(1, 0, 2557)-(2, 0, 0)","(2, 0, 5)-(1, 0, 286)","(2, 0, 25)-(2, 0, 1)","(1, 0, 2433)-(2, 0, 8)","(2, 0, 3)-(1, 0, 1012)","(2, 0, 43)-(1, 0, 2567)",...,"(2, 0, 14)-(1, 0, 1605)","(2, 0, 39)-(1, 0, 1295)","(2, 0, 1)-(1, 0, 477)","(1, 0, 2728)-(2, 0, 3)","(2, 0, 28)-(3, 0, 8)","(1, 0, 2771)-(2, 0, 14)","(2, 0, 37)-(1, 0, 152)","(2, 0, 31)-(2, 0, 2)","(2, 0, 40)-(1, 0, 2049)","(2, 0, 12)-(1, 0, 1276)"
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
kernel='linear'
X = edf.values
y = res_df['class'].values
clf = svm.SVC(decision_function_shape='ovo', kernel=kernel)
cvs = cross_val_score(clf, X, y, cv=10)

In [31]:
cvs.mean()

0.9049271513028001